In [12]:
import copy

SAMPLES = [
    '...81.....2........1.9..7...7..25.934.2............5...975.....563.....4......68.',
    '1..5.37..6.3..8.9......98...1.......8761..........6...........7.8.9.76.47...6.312',
    '..5...74.3..6...19.....1..5...7...2.9....58..7..84......3.9...2.9.4.....8.....1.3',
    '........5.2...9....9..2...373..481.....36....58....4...1...358...42.......978...2',
]
NUMBERS = '123456789'

In [13]:
def checkIfValid(puzzle):
    '''Check rows for duplicates,
    Check cols for duplicates,
    Check boxes for duplicates'''
    # Check rows
    for n in range(9):
        row = puzzle[n]
        if len(row) != len(set(row)):
            return False
    # Check cols
    for n in range(9):
        col = [puzzle[i][n] for i in range(9)]
        if len(col) != len(set(col)):
            return False
    # Check boxes
    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            box = [puzzle[i + x][j + y] for x in range(3) for y in range(3)]
            if len(box) != len(set(box)):
                return False
    return True

In [ ]:
def mostBasicDepthSearch(parent_puzzle, depth=0):
    '''Solve the puzzle'''
    # Make a unique copy of the puzzle so we don't modify paren't version
    puzzle = copy.deepcopy(parent_puzzle)

    # Find the first cell that is empty
    for row_num, row in enumerate(parent_puzzle):
        for col_num, cell in enumerate(row):
            # Skip known cells
            if cell != 0:
                continue
            
            row_set = set(row)
            col_set = set([puzzle[i][col_num] for i in range(9)])
            row_lower, row_upper = row_num//3*3, row_num//3*3+3
            col_lower, col_upper = col_num//3*3, col_num//3*3+3
            square_set = set([puzzle[i][j] for i in range(row_lower, row_upper) for j in range(col_lower, col_upper)])

            # Check for missing numbers
            combined_set = row_set.union(col_set).union(square_set)
            missing_set = set(NUMBERS).difference(combined_set)

            # Try each valid number
            for number in missing_set:
                puzzle[row_num][col_num] = number
                child_puzzle = mostBasicDepthSearch(puzzle, depth+1)
                if child_puzzle:
                    return child_puzzle


In [ ]:
def doTheThing(parent_puzzle, depth=0):
    '''Solve the puzzle'''
    # Make a unique copy of the puzzle so we don't modify paren't version
    puzzle = copy.deepcopy(parent_puzzle)

    # Compute the set of all possible numbers for each cell
    possible_numbers = [[set() for _ in range(9)] for _ in range(9)]
    for row_num, row in enumerate(parent_puzzle):
        for col_num, cell in enumerate(row):
            # Skip known cells
            if cell != 0:
                continue
            
            row_set = set(row)
            col_set = set([puzzle[i][col_num] for i in range(9)])
            row_lower, row_upper = row_num//3*3, row_num//3*3+3
            col_lower, col_upper = col_num//3*3, col_num//3*3+3
            square_set = set([puzzle[i][j] for i in range(row_lower, row_upper) 
                              for j in range(col_lower, col_upper)])

            # Check for missing numbers
            combined_set = row_set.union(col_set).union(square_set)
            missing_set = set(NUMBERS).difference(combined_set)
            possible_numbers[row_num][col_num] = missing_set
    
    # Check for naked singles
    for row_num, row in enumerate(possible_numbers):
        for col_num, possible in enumerate(row):
            if len(possible) == 1:
                # We found a naked single
                puzzle[row_num][col_num] = possible.pop()

    # Check for hidden singles
    for row_num, row in enumerate(possible_numbers):
        for col_num, possible in enumerate(row):
            # Check each number to see if it is not possible for each of...
            for number in possible:
                # Check the row
                row_set = set.union(*[possible_numbers[row_num][i] 
                                      for i in range(9) if i != col_num])
                if number not in row_set:
                    puzzle[row_num][col_num] = number
                    break
                # Check the col
                col_set = set.union(*[possible_numbers[i][col_num] 
                                        for i in range(9) if i != row_num])
                if number not in col_set:
                    puzzle[row_num][col_num] = number
                    break
                # Check the square
                row_lower, row_upper = row_num//3*3, row_num//3*3+3
                col_lower, col_upper = col_num//3*3, col_num//3*3+3
                square_set = set.union(*[possible_numbers[i][j] 
                                         for i in range(row_lower, row_upper) 
                                         for j in range(col_lower, col_upper) 
                                         if i != row_num and j != col_num])
                if number not in square_set:
                    puzzle[row_num][col_num] = number
                    break

    # Make a guess for a cell with the fewest possible numbers
    min_possible = 10
    min_loc = None
    for row_num, row in enumerate(possible_numbers):
        for col_num, possible in enumerate(row):
            if len(possible) == 0:
                continue
            if len(possible) < min_possible:
                min_possible = len(possible)
                min_loc = (row_num, col_num)

    


In [ ]:
# test_function = mostBasicDepthSearch
test_function = doTheThing

for sample in SAMPLES:
    print(sample)
    sample = list(sample)
    sample = [int(n) if n in NUMBERS else 0 for n in sample]
    # Convert it to a 2D list
    puzzle = [sample[i:i+9] for i in range(0, 81, 9)]
    for row in puzzle:
        print(row)
    print()
    solution = test_function(puzzle)
    for row in solution:
        print(row)
    is_valid = checkIfValid(solution)
    print('Valid:', is_valid)
    print()

...81.....2........1.9..7...7..25.934.2............5...975.....563.....4......68.
[0, 0, 0, 8, 1, 0, 0, 0, 0]
[0, 2, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 9, 0, 0, 7, 0, 0]
[0, 7, 0, 0, 2, 5, 0, 9, 3]
[4, 0, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 5, 0, 0]
[0, 9, 7, 5, 0, 0, 0, 0, 0]
[5, 6, 3, 0, 0, 0, 0, 0, 4]
[0, 0, 0, 0, 0, 0, 6, 8, 0]



KeyboardInterrupt: 